# Performance Comparison of Sketch Algorithms

This notebook compares the performance and accuracy of different probabilistic data structures implemented in the sketches library.

In [ ]:
import sketches
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import time
from collections import defaultdict

## Accuracy Comparison

Let's compare the accuracy of different sketch algorithms across various cardinalities.

In [ ]:
def test_accuracy(sketch_class, cardinalities, lg_k=12, k=4096):
    results = []
    
    for n in cardinalities:
        if 'Theta' in sketch_class.__name__:
            sketch = sketch_class(k)
        elif 'Cpc' in sketch_class.__name__:
            sketch = sketch_class(lg_k)
        else:
            sketch = sketch_class(lg_k)
            
        # Add n unique elements
        for i in range(n):
            sketch.update(f"item_{i}")
            
        estimate = sketch.estimate()
        error = abs(estimate - n) / n
        results.append({'cardinality': n, 'estimate': estimate, 'error': error})
        
    return results

In [ ]:
# Test cardinalities
cardinalities = [100, 500, 1000, 5000, 10000, 50000, 100000]

# Test different sketch types
sketch_classes = [
    sketches.HllSketch,
    sketches.CpcSketch,
    sketches.ThetaSketch
]

accuracy_results = {}
for sketch_class in sketch_classes:
    print(f"Testing {sketch_class.__name__}...")
    accuracy_results[sketch_class.__name__] = test_accuracy(sketch_class, cardinalities)

In [ ]:
# Plot accuracy results
plt.figure(figsize=(12, 8))

for sketch_name, results in accuracy_results.items():
    df = pl.DataFrame(results)
    cardinality_data = df['cardinality'].to_numpy()
    error_data = df['error'].to_numpy()
    plt.plot(cardinality_data, error_data, marker='o', label=sketch_name, linewidth=2)

plt.xlabel('True Cardinality')
plt.ylabel('Relative Error')
plt.title('Accuracy Comparison of Sketch Algorithms')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.yscale('log')
plt.show()

## Performance Comparison

Now let's compare the update performance of different algorithms.

In [ ]:
def benchmark_updates(sketch_class, n_updates=100000, lg_k=12, k=4096):
    if 'Theta' in sketch_class.__name__:
        sketch = sketch_class(k)
    elif 'Cpc' in sketch_class.__name__:
        sketch = sketch_class(lg_k)
    else:
        sketch = sketch_class(lg_k)
    
    start_time = time.time()
    
    for i in range(n_updates):
        sketch.update(f"item_{i}")
    
    end_time = time.time()
    
    return {
        'total_time': end_time - start_time,
        'updates_per_second': n_updates / (end_time - start_time),
        'estimate': sketch.estimate()
    }

In [ ]:
# Benchmark performance
n_updates = 100000
performance_results = {}

for sketch_class in sketch_classes:
    print(f"Benchmarking {sketch_class.__name__}...")
    performance_results[sketch_class.__name__] = benchmark_updates(sketch_class, n_updates)

# Display results
perf_data = []
for name, results in performance_results.items():
    perf_data.append({
        'sketch': name,
        'total_time': results['total_time'],
        'updates_per_second': results['updates_per_second'],
        'estimate': results['estimate']
    })

perf_df = pl.DataFrame(perf_data)
print("\nPerformance Results:")
print(perf_df)

In [ ]:
# Plot performance comparison
plt.figure(figsize=(10, 6))
sketch_names = list(performance_results.keys())
updates_per_sec = [performance_results[name]['updates_per_second'] for name in sketch_names]

bars = plt.bar(sketch_names, updates_per_sec, color=['blue', 'orange', 'green', 'red'])
plt.xlabel('Sketch Algorithm')
plt.ylabel('Updates per Second')
plt.title('Performance Comparison: Updates per Second')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, value in zip(bars, updates_per_sec):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + bar.get_height()*0.01, 
             f'{value:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Memory Usage Comparison

Let's compare the memory footprint of different sketch implementations.

In [ ]:
def get_sketch_size(sketch_class, lg_k=12, k=4096):
    if 'Theta' in sketch_class.__name__:
        sketch = sketch_class(k)
    elif 'Cpc' in sketch_class.__name__:
        sketch = sketch_class(lg_k)
    else:
        sketch = sketch_class(lg_k)
    
    # Add some data
    for i in range(10000):
        sketch.update(f"item_{i}")
    
    # Get serialized size as proxy for memory usage
    # Note: ThetaSketch doesn't have to_bytes method, so we estimate size
    if hasattr(sketch, 'to_bytes'):
        serialized = sketch.to_bytes()
        return len(serialized)
    else:
        # Estimate size for sketches without serialization
        if 'Theta' in sketch_class.__name__:
            return sketch.sample_capacity() * 8  # Rough estimate: 8 bytes per entry
        else:
            return 4096  # Default estimate

memory_usage = {}
for sketch_class in sketch_classes:
    memory_usage[sketch_class.__name__] = get_sketch_size(sketch_class)

# Plot memory usage
plt.figure(figsize=(10, 6))
sketch_names = list(memory_usage.keys())
sizes = list(memory_usage.values())

bars = plt.bar(sketch_names, sizes, color=['blue', 'orange', 'green'])
plt.xlabel('Sketch Algorithm')
plt.ylabel('Estimated Size (bytes)')
plt.title('Memory Usage Comparison')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, value in zip(bars, sizes):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + bar.get_height()*0.01, 
             f'{value}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nMemory Usage (bytes):")
for name, size in memory_usage.items():
    print(f"{name}: {size:,} bytes")

## Conclusion

This notebook demonstrates the trade-offs between different sketch algorithms:

- **HyperLogLog**: Good balance of accuracy and performance
- **HyperLogLog++**: Improved accuracy for small cardinalities
- **CPC**: Compressed representation with good accuracy
- **Theta Sketch**: Set operations support with configurable accuracy

Choose the algorithm based on your specific requirements for accuracy, performance, and memory usage.